### Проверка работоспособности и качества пайплайна
Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [13]:
import pandas as pd
import dill
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import recall_score, precision_score, roc_auc_score, roc_curve #,scorer
from sklearn.metrics import f1_score

In [14]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [15]:
X_test.head(3)

,comment,toxic
0,безродный потомок холопа рассуждает о национал...,1.0
1,"Интересная тема, однако. ОТП вроде венгерский ...",0.0
2,"Суть требования про В30 - не прочность, а водо...",0.0


In [16]:
with open('gbc_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [17]:
predictions = pipeline.predict_proba(X_test.iloc[:500])
pd.DataFrame({'preds': predictions[:, 1]}).to_csv("test_predictions.csv", index=None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
roc_auc_score(y_score=predictions[:, 1][:], y_true=y_test.iloc[:500])

0.8205141996557659

Ну и другие метрики посмотрю

In [19]:
preds = pipeline.predict(X_test)

In [20]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return f1, roc, rec, prc

In [22]:
score = evaluate_results(y_test, preds)

Classification results:
f1: 45.94%
roc: 64.46%
recall: 31.24%
precision: 86.81%
